In [1]:
# import torch
import pickle
import os
import numpy as np
from scipy.spatial.transform import Rotation as R

def compute_global_rotation(pose_axis_anges, joint_idx):
    """
    calculating joints' global rotation
    Args:
        pose_axis_anges (np.array): SMPLX's local pose (22,3)
    Returns:
        np.array: (3, 3)
    """
    global_rotation = np.eye(3)
    parents = [-1,  0,  0,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9,  9,  9, 12, 13, 14, 16, 17, 18, 19]
    while joint_idx != -1:
        joint_rotation = R.from_rotvec(pose_axis_anges[joint_idx]).as_matrix()
        global_rotation = joint_rotation @ global_rotation
        joint_idx = parents[joint_idx]
    return global_rotation

M = np.diag([-1, 1, 1])     

# GVHMR data

In [217]:

# file_gvhmr = r"D:\AI_Teste\GVHMR\pkl_anim\burst_kick.pkl"
file_gvhmr = r"D:\AI\hamer\Testes\SignLanguage\sign_language.pkl"
with(open(file_gvhmr, "rb")) as f:
    pkl_gvhmr = pickle.load(f)


In [218]:

smplx_params = {}
smplx_params["global_orient"] = pkl_gvhmr["smpl_params_global"]["global_orient"]
smplx_params["body_pose"] = pkl_gvhmr["smpl_params_global"]["body_pose"]
smplx_params["transl"] = pkl_gvhmr["smpl_params_global"]["transl"]

# PromptHMR data

In [223]:
file_phmr = r"D:\AI\hamer\Testes\SignLanguage\phmr_sign_language_results_world_blender.pkl"

with(open(file_phmr, "rb")) as f:
    pkl_phmr = pickle.load(f)

In [224]:
num_phmr_pessoa = 0
smplx_params = {}
smplx_params["global_orient"] =pkl_phmr[num_phmr_pessoa]["poses"][:,:3] # o [0] é necessario ac que para selecionar a pessoa do phmr
smplx_params["body_pose"] = pkl_phmr[num_phmr_pessoa]["poses"][:,3:66]
smplx_params["transl"] = pkl_phmr[num_phmr_pessoa]["trans"]

# Hamer data

## Pkl sequence Convertendo em apenas um arquivo com tudo

In [219]:
import glob
file_hamer = r"D:\AI\hamer\Testes\SignLanguage\hammer\result_000_*.pkl"
# file_hamer = r"D:\AI\hamer\fight_video\hamer\result_000_*.pkl"

list_hamer_pkl = glob.glob(file_hamer)

# with(open(file_hamer, "rb")) as f:
#     pkl_hamer = pickle.load(f)

In [226]:
# checando maximo de pessoas a considerar
# list_people =[]
max_people = 0
num_people = 0
max_lh = 0
max_rh = 0
for i, file in enumerate(list_hamer_pkl): # primeiro loop para poder encontrar os tamanhos de arrays a criar
    with(open(file, "rb")) as f:
        pkl_hamer = pickle.load(f)
    frame = int(os.path.splitext((os.path.basename(file)))[0].split('_')[-1])
    # print('frames: ',frame, ' - personid: ',pkl_hamer["personid"],' - right: ',pkl_hamer["right"], 'poses: ',pkl_hamer["hand_pose"].shape)
    # list_people.append([i,pkl_hamer["personid"],pkl_hamer["right"]])
    num_people = len(pkl_hamer["personid"])
    if num_people > max_people:
        max_people = num_people
    
    num_lh = np.count_nonzero(pkl_hamer["right"] == 0)
    if num_lh > max_lh:
        max_lh = num_lh
    num_rh = np.count_nonzero(pkl_hamer["right"] == 1)
    if num_rh > max_rh:
        max_rh = num_rh


print('frames: ',frame,'max_people: ',max_people, 'max_lh: ',max_lh, 'max_rh: ',max_rh)



frames:  442 max_people:  2 max_lh:  1 max_rh:  1


In [227]:
# Criando pose numpy array com tamanho ja definido
lh_pose = np.zeros((frame+1,max_lh,15,3,3))
rh_pose = np.zeros((frame+1,max_rh,15,3,3))
lh_global_orient = np.zeros((frame+1,max_lh,1 ,3,3))
rh_global_orient = np.zeros((frame+1,max_rh,1 ,3,3))
hand_export={}
for i, file in enumerate(list_hamer_pkl): # segundo loop para poder encontrar os tamanhos de arrays a criar
    with(open(file, "rb")) as f:
        pkl_hamer = pickle.load(f)
    fr = int(os.path.splitext((os.path.basename(file)))[0].split('_')[-1])
    in_lf = 0 #contar quantidade de vezes que entrou no lf
    in_rh = 0
    for go in range(pkl_hamer["right"].shape[0]):
        if pkl_hamer["right"][go] == 0:
            #trabalhando no left_hand
            in_lf += 1
            # print('go: ',go,'in_lf: ',in_lf,'in_rh: ',in_rh,'frame: ',fr)
            # print('go-in_rh: ',go-in_rh)
            lh_pose[fr][go-in_rh] = pkl_hamer["hand_pose"][go]
            lh_global_orient[fr][go-in_rh] = pkl_hamer["global_orient"][go]
        else:
            #trabalhando no right_hand
            in_rh += 1
            # print('go: ',go,'in_lf: ',in_lf,'in_rh: ',in_rh,'frame: ',fr)
            # print('go-in_lf: ',go-in_lf)
            rh_pose[fr][go-in_lf] = pkl_hamer["hand_pose"][go]
            rh_global_orient[fr][go-in_lf] = pkl_hamer["global_orient"][go]


hand_export["lh_pose"] = lh_pose
hand_export["rh_pose"] = rh_pose
hand_export["lh_global_orient"] = lh_global_orient
hand_export["rh_global_orient"] = rh_global_orient

pkl_out = os.path.join(os.path.dirname(file_hamer),"_Hammer_Final_to_convert.pkl")
with(open(pkl_out, "wb")) as f:
    pickle.dump(hand_export, f)

# Teste wilor

In [25]:
file_wilor = r"D:\AI\wilor\WiLoR\demo_out\result_000_00091.pkl"
# file_wilor = r"D:\AI\wilor\WiLoR\demo_out\batch_000_00091.pkl"
with(open(file_wilor, "rb")) as f:
    pkl_wilor = pickle.load(f)

In [24]:
pkl_wilor.keys()

dict_keys(['personid', 'right'])

In [26]:
pkl_wilor["hand_pose"].shape
# pkl_wilor["right"]

(2, 15, 3, 3)

In [42]:
test.shape

(2, 4)

In [30]:
test[0].shape

(4, 15, 3, 3)

In [46]:
hamer_mano_params = {}
hamer_mano_params["global_orient"] = pkl_hamer["global_orient"]
hamer_mano_params["hand_pose"] = pkl_hamer["hand_pose"]

# Code Convert com loop

## simngle hamer test

In [ ]:
# tentando montar com apenas 1 hand shape
# file_single_hamer = r"D:\AI\hamer\burst_kick_prompthmr_hamer\hamer\result_000_00000.pkl"
# with(open(file_single_hamer, "rb")) as f:
#     single_pkl_hamer = pickle.load(f)


## resto do codigo com loop

In [228]:
import numpy as np
qtd = smplx_params["global_orient"].shape[0]
# qtd=55
M = np.diag([-1, 1, 1])                                                                                                              # Preparing for the left hand switch

lh_num = 0
rh_num = 0

export_body_hand = {}
export_body_hand["global_orient"] = np.zeros((qtd,3))
export_body_hand["body_pose"] = np.zeros((qtd,63))

export_body_hand["left_hand_pose"] = np.zeros((qtd,45))
export_body_hand["right_hand_pose"] = np.zeros((qtd,45))


# lh_pose_final = np.zeros((45))
# rh_pose_final = np.zeros((45))
body_pose_final = np.zeros((63))




for fra in range(qtd):
    # print('frame: ',fra)
    # Assuming that your data are stored in gvhmr_smplx_params and hamer_mano_params
    # full_body_pose = torch.concatenate((gvhmr_smplx_params["global_orient"][0], gvhmr_smplx_params["body_pose"][0].reshape(21, 3)), dim=0)     # gvhmr_smplx_params["global_orient"]: (3, 3)
    # full_body_pose = np.concatenate((smplx_params["global_orient"][1].reshape(-1,3), smplx_params["body_pose"][1].reshape(21, 3)))     
    full_body_pose = np.concatenate((smplx_params["global_orient"][fra].reshape(-1,3), smplx_params["body_pose"][fra].reshape(-1, 3)))     
    left_elbow_global_rot = compute_global_rotation(full_body_pose, 18) # left elbow IDX: 18
    right_elbow_global_rot = compute_global_rotation(full_body_pose, 19) # left elbow IDX: 19

    # left_wrist_global_rot = hamer_mano_params["global_orient"][0]       #### original apenas para Referencia                                                   # hamer_mano_params["global_orient"]: (2, 3, 3)
    left_wrist_global_rot = lh_global_orient[fra][lh_num][0]                                                                     #lh_global_orient : (frame,  pessoa lh, 1, 3, 3)
    # left_wrist_global_rot = single_pkl_hamer['global_orient'][0]
    left_wrist_global_rot = M @ left_wrist_global_rot @ M                                                                                # mirror switch
    left_wrist_pose = np.linalg.inv(left_elbow_global_rot) @ left_wrist_global_rot

    right_wrist_global_rot = rh_global_orient[fra][rh_num][0]
    # right_wrist_global_rot = single_pkl_hamer['global_orient'][1]
    right_wrist_pose = np.linalg.inv(right_elbow_global_rot) @ right_wrist_global_rot

    last_lw_non_zero_pose = np.eye(3)
    last_rw_non_zero_pose = np.eye(3)

    if (left_wrist_pose == 0).all():
        left_wrist_pose = last_lw_non_zero_pose
    else:
        last_lw_non_zero_pose = left_wrist_pose
    left_wrist_pose_vec = R.from_matrix(left_wrist_pose).as_rotvec()

    if (right_wrist_pose == 0).all():
        right_wrist_pose = last_rw_non_zero_pose
    else:
        last_rw_non_zero_pose = right_wrist_pose
    right_wrist_pose_vec = R.from_matrix(right_wrist_pose).as_rotvec()



    left_hand_pose = np.ones(45)
    right_hand_pose = np.ones(45)
    for i in range(15):
        # left_finger_pose = M @ hamer_mano_params["hand_pose"][0][i] @ M                                                    # hamer_mano_params["hand_pose"]: (2, 15, 3, 3)
        ########## Left fingers
        lh_pose_use = lh_pose[fra][lh_num][i]
        # lh_pose_use = single_pkl_hamer['hand_pose'][0][i] ## estatico para teste
        tmp_lh_pose = np.eye(3)
        if (lh_pose_use == 0).all():
            lh_pose_use = tmp_lh_pose
        else:
            tmp_lh_pose = lh_pose_use

        left_finger_pose = M @ lh_pose_use @ M                                                    # lh_pose: (222, 1, 15, 3, 3) (frames,pessoa lh,15,3,3)
        left_finger_pose_vec = R.from_matrix(left_finger_pose).as_rotvec()
        left_hand_pose[i*3: i*3+3] = left_finger_pose_vec
        
        ########### Right Fingers
        rh_pose_use = rh_pose[fra][rh_num][i]
        # rh_pose_use = single_pkl_hamer['hand_pose'][1][i] ### estatico para teste
        tmp_rh_pose = np.eye(3)
        if (rh_pose_use == 0).all():
            rh_pose_use = tmp_rh_pose
        else:
            tmp_rh_pose = rh_pose_use

        right_finger_pose = rh_pose_use #hamer_mano_params["hand_pose"][1][i]
        right_finger_pose_vec = R.from_matrix(right_finger_pose).as_rotvec()
        right_hand_pose[i*3: i*3+3] = right_finger_pose_vec
    



    body_pose_final[:63] = smplx_params["body_pose"][fra] #fazendo uma copia com um array ja no tamanho esperado esta limi
    # body_pose_final[57: 60] = left_wrist_pose_vec #trocando a direcao do pulso
    # body_pose_final[60: 63] = right_wrist_pose_vec



    # lh_pose_final = left_hand_pose
    # rh_pose_final = right_hand_pose

    # gvhmr_smplx_params["body_pose"][0][57: 60] = left_wrist_pose_vec
    # gvhmr_smplx_params["body_pose"][0][60: 63] = right_wrist_pose_vec
    # gvhmr_smplx_params["left_hand_pose"] = left_hand_pose
    # gvhmr_smplx_params["right_hand_pose"] = right_hand_pose
    export_body_hand["global_orient"][fra] = smplx_params["global_orient"][fra]
    export_body_hand["body_pose"][fra] = body_pose_final
    export_body_hand["left_hand_pose"][fra] = left_hand_pose
    export_body_hand["right_hand_pose"][fra] = right_hand_pose
export_body_hand["transl"] = smplx_params["transl"] 

# pkl_out = os.path.join(os.path.dirname(file_hamer),"_Hammer_Final_com_mao.pkl")
pkl_out = os.path.join(os.path.dirname(file_hamer),"_PHMR_Hammer_Final_com_mao.pkl")
# pkl_out = os.path.join(os.path.dirname(file_hamer),"_GVHMR_Hammer_Final_com_mao.pkl")
with(open(pkl_out, "wb")) as f:
    pickle.dump(export_body_hand, f)

In [129]:
smplx_params["body_pose"][fra].shape

(63,)

In [170]:
body_pose_final[63: 66]
# right_wrist_pose_vec

array([], dtype=float32)

# code to convert (antigo, sem loop)

In [202]:
import numpy as np
qtd = pkl_phmr[0]["trans"].shape[0]

M = np.diag([-1, 1, 1])                                                                                                              # Preparing for the left hand switch

# Assuming that your data are stored in gvhmr_smplx_params and hamer_mano_params
# full_body_pose = torch.concatenate((gvhmr_smplx_params["global_orient"][0], gvhmr_smplx_params["body_pose"][0].reshape(21, 3)), dim=0)     # gvhmr_smplx_params["global_orient"]: (3, 3)
full_body_pose = np.concatenate((smplx_params["global_orient"][0].reshape(-1,3), smplx_params["body_pose"][0].reshape(21, 3)))     # gvhmr_smplx_params["global_orient"]: (3, 3)
left_elbow_global_rot = compute_global_rotation(full_body_pose, 18) # left elbow IDX: 18
right_elbow_global_rot = compute_global_rotation(full_body_pose, 19) # left elbow IDX: 19

left_wrist_global_rot = single_pkl_hamer["global_orient"][0]                                                          # hamer_mano_params["global_orient"]: (2, 3, 3)
left_wrist_global_rot = M @ left_wrist_global_rot @ M                                                                                # mirror switch
left_wrist_pose = np.linalg.inv(left_elbow_global_rot) @ left_wrist_global_rot
# left_wrist_pose = left_wrist_global_rot

right_wrist_global_rot = single_pkl_hamer["global_orient"][1]
right_wrist_pose = np.linalg.inv(right_elbow_global_rot) @ right_wrist_global_rot
# right_wrist_pose = right_wrist_global_rot

left_wrist_pose_vec = R.from_matrix(left_wrist_pose).as_rotvec()
right_wrist_pose_vec = R.from_matrix(right_wrist_pose).as_rotvec()

left_hand_pose = np.ones(45)
right_hand_pose = np.ones(45)
for i in range(15):
    left_finger_pose = M @ single_pkl_hamer["hand_pose"][0][i] @ M                                                    # hamer_mano_params["hand_pose"]: (2, 15, 3, 3)
    left_finger_pose_vec = R.from_matrix(left_finger_pose).as_rotvec()
    left_hand_pose[i*3: i*3+3] = left_finger_pose_vec
    
    right_finger_pose = single_pkl_hamer["hand_pose"][1][i]
    right_finger_pose_vec = R.from_matrix(right_finger_pose).as_rotvec()
    right_hand_pose[i*3: i*3+3] = right_finger_pose_vec

# smplx_params["body_pose"][0][57: 60] = left_wrist_pose_vec
# smplx_params["body_pose"][0][60: 63] = right_wrist_pose_vec
smplx_params["left_hand_pose"] = left_hand_pose
smplx_params["right_hand_pose"] = right_hand_pose

pkl_out = r"D:\AI\hamer\sample_gvhmr_and_hamer\gvhmr_with_hammer.pkl"
with(open(pkl_out, "wb")) as f:
    pickle.dump(smplx_params, f)